In [8]:
import queue
import threading
import concurrent.futures
import time

class PriorityMessageQueue:
    def __init__(self):
        self.queue = queue.PriorityQueue()
        self.condition = threading.Condition()

    def enqueue_message(self, message):
        with self.condition:
            _, priority, _ = message
            self.queue.put((priority, message))
            self.condition.notify()

    def dequeue_message(self):
        with self.condition:
            while self.queue.empty():
                self.condition.wait()
            return self.queue.get()[1]

    def peek_message(self):
        with self.condition:
            while self.queue.empty():
                self.condition.wait()
            return self.queue.queue[0][1]

    def empty(self):
        with self.condition:
            return self.queue.empty()

class ThreadPool:
    def __init__(self, num_threads):
        self.executor = concurrent.futures.ThreadPoolExecutor(max_workers=num_threads)
        self.lock = threading.Lock()

    def submit_task(self, task, *args, **kwargs):
        with self.lock:
            return self.executor.submit(task, *args, **kwargs)

def simple_action(message):
    print(f"Executing simple action: {message}")

def send_message(sender_id, receiver_id, priority, message, message_queue, thread_pools):
    print(f"Thread {sender_id} sending message to Thread {receiver_id} with priority {priority}: {message}")
    message_queue.enqueue_message((receiver_id, priority, message))

    if thread_pools[receiver_id].submit_task(process_message, receiver_id, message_queue, thread_pools[receiver_id]):
        pass

def process_message(thread_id, message_queue, thread_pool):
    while not message_queue.empty():
        message = message_queue.dequeue_message()
        if message:
            receiver_id, _, message_content = message
            print(f"Thread {thread_id} processing message from Thread {receiver_id}: {message_content}")
            thread_pool.submit_task(simple_action, message_content)

def main():
    num_threads = 3
    message_queue = PriorityMessageQueue()
    thread_pools = [ThreadPool(1) for _ in range(num_threads)]

    threads = []
    for i in range(num_threads):
        thread = threading.Thread(target=process_message, args=(i, message_queue, thread_pools[i]))
        thread.start()
        threads.append(thread)

    # Sending messages
    send_message(0, 1, 2, "Hello from Thread 0 to Thread 1!", message_queue, thread_pools)
    send_message(1, 2, 1, "Hi from Thread 1 to Thread 2!", message_queue, thread_pools)
    send_message(2, 0, 0, "Hey from Thread 2 to Thread 0!", message_queue, thread_pools)

    # Allow threads to finish processing
    time.sleep(2)

    # Shutting down thread pool
    for pool in thread_pools:
        pool.executor.shutdown()

    # Wait for threads to finish
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    main()

Thread 0 sending message to Thread 1 with priority 2: Hello from Thread 0 to Thread 1!
Thread 1 processing message from Thread 1: Hello from Thread 0 to Thread 1!
Thread 1 sending message to Thread 2 with priority 1: Hi from Thread 1 to Thread 2!
Thread 2 processing message from Thread 2: Hi from Thread 1 to Thread 2!
Thread 2 sending message to Thread 0 with priority 0: Hey from Thread 2 to Thread 0!
Thread 1 processing message from Thread 0: Hey from Thread 2 to Thread 0!
Executing simple action: Hi from Thread 1 to Thread 2!
Executing simple action: Hello from Thread 0 to Thread 1!
Executing simple action: Hey from Thread 2 to Thread 0!
